In [19]:
import numpy as np
import pandas as pd

In [20]:
'''np.random.randn(y, 1):
For each layer y (number of neurons in that layer), this generates a column vector of shape (y, 1) with random values sampled from a standard normal distribution (mean = 0, standard deviation = 1).
Each value in this vector represents the initial bias for a neuron in that layer.'''

'''
List comprehension:
The list comprehension [np.random.randn(y, 1) for y in sizes[1:]] iterates over the number of neurons in each layer (excluding the input layer) and creates a bias vector for each layer.
'''
'''
Example:
If sizes = [784, 30, 10] (e.g., for an MNIST network with 784 input neurons, 30 hidden neurons, and 10 output neurons):

sizes[1:] = [30, 10]
self.biases will be a list containing:
A (30, 1) array for the biases of the hidden layer.
A (10, 1) array for the biases of the output layer.
'''

'''
Example:
If sizes = [784, 30, 10] (e.g., for an MNIST network with 784 input neurons, 30 hidden neurons, and 10 output neurons):

sizes[:-1] = [784, 30] (input and hidden layers).
sizes[1:] = [30, 10] (hidden and output layers).
zip(sizes[:-1], sizes[1:]) = [(784, 30), (30, 10)].
The resulting self.weights will be a list containing:

A (30, 784) matrix for the weights connecting the input layer to the hidden layer.
A (10, 30) matrix for the weights connecting the hidden layer to the output layer.
'''

"""Train the neural network using mini-batch stochastic gradient descent. The "training_data" is a list of tuples "(x, y)" representing the training inputs and the desired outputs.
If "test_data" is provided then the network will be evaluated against the test data after each epoch, and partial progress printed out.  This is useful for tracking progress, but slows 
things down substantially. The training_data is a list of tuples (x, y) representing the training inputs and corresponding desired outputs."""

"""Update the network's weights and biases by applying gradient descent using backpropagation to a single mini batch. The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
is the learning rate."""

"""Return a tuple ``(nabla_b, nabla_w)`` representing the
gradient for the cost function C_x.  ``nabla_b`` and
``nabla_w`` are layer-by-layer lists of numpy arrays, similar
to ``self.biases`` and ``self.weights``."""

"""Return the number of test inputs for which the neural
network outputs the correct result. Note that the neural
network's output is assumed to be the index of whichever
neuron in the final layer has the highest activation."""

"""Return the vector of partial derivatives \partial C_x /
\partial a for the output activations."""

<>:49: SyntaxWarning: invalid escape sequence '\p'
<>:49: SyntaxWarning: invalid escape sequence '\p'
C:\Users\getch\AppData\Local\Temp\ipykernel_19108\2973414786.py:49: SyntaxWarning: invalid escape sequence '\p'
  """Return the vector of partial derivatives \partial C_x /


'Return the vector of partial derivatives \\partial C_x /\n\\partial a for the output activations.'

The randn function in NumPy (np.random.randn) generates samples from a standard normal distribution (Gaussian distribution) with a mean of 0 and a standard deviation of 1.

In [21]:
# train_set, valid_set, test_set = pd.read_pickle("mnist.pkl.gz")

In [22]:
# train_set

In [23]:
# train_x, train_y = train_set

In [24]:
# train_x

In [25]:
# train_y

In [26]:
# valid_set

In [27]:
# val_x, val_y = valid_set

In [28]:
# val_x

In [29]:
# val_y

In [30]:
# test_x, test_y = test_set

In [31]:
# test_x

In [32]:
# test_y

In [33]:
# import network

# net = network.Network([784, 30, 10])
# net

In [34]:
# net.SGD(train_x, 30, 10, 3.0, test_data=test_x)

In [35]:
import mnist_loader
from network import Network

# Load data
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

# Initialize network and train
net = Network([784, 100, 10]) #net = Network([784, 30, 10])

sizes: [784, 100, 10], biases: [array([[-1.78078262],
       [-0.89016162],
       [-1.19155658],
       [ 0.85194833],
       [-0.27432219],
       [-0.25975053],
       [-0.39474725],
       [-0.50358384],
       [-0.43094787],
       [ 1.27623122],
       [ 1.19438192],
       [-0.16455378],
       [-0.28056151],
       [ 0.04296755],
       [ 0.25981361],
       [-0.40040335],
       [-0.77458352],
       [-1.24124419],
       [ 1.07439232],
       [-0.21151755],
       [ 0.18418705],
       [ 1.02252436],
       [ 1.38113918],
       [-1.27458674],
       [ 0.24879042],
       [-0.71909077],
       [-0.63470478],
       [-0.13111967],
       [-1.16846243],
       [-0.37624491],
       [-0.28139207],
       [-0.5303361 ],
       [-0.00423171],
       [ 0.19126184],
       [ 2.11163065],
       [ 1.19943792],
       [ 0.48521183],
       [-0.9591942 ],
       [-1.48272443],
       [ 0.32293608],
       [ 0.73620826],
       [-2.78489654],
       [-0.31192295],
       [-1.22857145],


In [36]:
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 5745 / 10000, took 8.52 seconds
Epoch 1: 5861 / 10000, took 8.47 seconds
Epoch 2: 5831 / 10000, took 8.28 seconds
Epoch 3: 6781 / 10000, took 8.27 seconds
Epoch 4: 6788 / 10000, took 8.15 seconds
Epoch 5: 6835 / 10000, took 8.79 seconds
Epoch 6: 6848 / 10000, took 8.97 seconds
Epoch 7: 6843 / 10000, took 8.29 seconds
Epoch 8: 6858 / 10000, took 8.00 seconds
Epoch 9: 6937 / 10000, took 8.23 seconds
Epoch 10: 6908 / 10000, took 8.54 seconds
Epoch 11: 7151 / 10000, took 8.54 seconds
Epoch 12: 7747 / 10000, took 8.55 seconds
Epoch 13: 7760 / 10000, took 8.28 seconds
Epoch 14: 8634 / 10000, took 8.68 seconds
Epoch 15: 8661 / 10000, took 8.78 seconds
Epoch 16: 8688 / 10000, took 9.11 seconds
Epoch 17: 8679 / 10000, took 8.76 seconds
Epoch 18: 8675 / 10000, took 8.52 seconds
Epoch 19: 8692 / 10000, took 8.16 seconds
Epoch 20: 8694 / 10000, took 8.93 seconds
Epoch 21: 8701 / 10000, took 8.68 seconds
Epoch 22: 8703 / 10000, took 8.33 seconds
Epoch 23: 8703 / 10000, took 8.66 seconds
Ep

In [37]:
net = Network([784, 30, 10])

sizes: [784, 30, 10], biases: [array([[-0.93116942],
       [ 0.19576305],
       [ 0.51322485],
       [-0.71239127],
       [-0.31445313],
       [ 0.16698926],
       [ 0.85876066],
       [-1.51907219],
       [ 1.57623259],
       [ 1.06074594],
       [ 1.08038854],
       [ 0.0204301 ],
       [-0.41960722],
       [-1.21862619],
       [ 0.47943708],
       [-1.74941566],
       [-0.94369981],
       [ 0.93916544],
       [ 0.7514525 ],
       [ 1.99300911],
       [ 2.29908134],
       [-1.00864668],
       [ 0.06250526],
       [-1.07136379],
       [ 1.62879467],
       [-1.27432364],
       [ 0.73715673],
       [ 1.16565282],
       [ 0.72375291],
       [-0.82660718]]), array([[-1.09194447],
       [ 1.30599739],
       [ 0.35339084],
       [ 0.75499937],
       [-1.11142724],
       [ 0.37506852],
       [ 0.79934151],
       [-0.35207666],
       [ 1.6000806 ],
       [-2.04447881]])], weights: [array([[ 0.33987128,  1.79190775, -2.28174759, ...,  0.06011286,
         

In [38]:
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 8992 / 10000, took 4.80 seconds
Epoch 1: 9184 / 10000, took 4.51 seconds
Epoch 2: 9295 / 10000, took 4.42 seconds
Epoch 3: 9300 / 10000, took 4.41 seconds
Epoch 4: 9380 / 10000, took 4.82 seconds
Epoch 5: 9385 / 10000, took 4.84 seconds
Epoch 6: 9401 / 10000, took 4.67 seconds
Epoch 7: 9398 / 10000, took 5.44 seconds
Epoch 8: 9403 / 10000, took 4.74 seconds
Epoch 9: 9448 / 10000, took 5.09 seconds
Epoch 10: 9411 / 10000, took 4.78 seconds
Epoch 11: 9464 / 10000, took 5.13 seconds
Epoch 12: 9459 / 10000, took 5.06 seconds
Epoch 13: 9464 / 10000, took 4.95 seconds
Epoch 14: 9483 / 10000, took 4.74 seconds
Epoch 15: 9466 / 10000, took 4.80 seconds
Epoch 16: 9475 / 10000, took 4.58 seconds
Epoch 17: 9446 / 10000, took 4.62 seconds
Epoch 18: 9485 / 10000, took 4.63 seconds
Epoch 19: 9472 / 10000, took 4.54 seconds
Epoch 20: 9480 / 10000, took 4.50 seconds
Epoch 21: 9496 / 10000, took 4.58 seconds
Epoch 22: 9505 / 10000, took 4.50 seconds
Epoch 23: 9491 / 10000, took 4.78 seconds
Ep